# MOVIE 2 SCRIPTS

Réalisation des scripts python pour movie2: création de la database, création des tables et insertions des données

In [1]:
import os  # pour utiliser les variables d'environnements

In [2]:
import psycopg2   # Le pendant de sqlite 3 pour postgresql

# création de la base de données


In [ ]:
# Code python pour créer la base de donnée sous postgresql

def ouvrir_connection(base_de_donnee, utilisateur, mot_passe, host='localhost', port=5432):
    
    try:
        # création de l'objet conn
        conn = psycopg2.connect(dbname= base_de_donnee, user=utilisateur, password = mot_passe, host="localhost", port= 5432)

        conn.set_session(autocommit=True)   # on force la sauvegarde
        
        # initialisation du cursor
        cursor = conn.cursor()
        
        # création de la base de donnée
        cursor.execute("CREATE DATABASE IF NOT EXISTS movie2")

        # gestion des erreurs
    except psycopg2.Error as e:
        print("Problème lors de la connection à la base de donnée")
        print(e)
        return None
        
        # actions a effectuées dans tous les cas, fermer la connection au serveur
    finally:
        cursor.close
        conn.close
    return conn
    

In [ ]:
# paramètres de connection
base_de_donnee = "postgres"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_pw')
    

In [ ]:
# exécution de la fonction connection
ouvrir_connection(base_de_donnee, utilisateur, mot_passe)

------------------

# Connection et création table

In [3]:
# import des librairies csv et pathlib
import csv
from pathlib import Path

# Répertoire dataset (small)
repertoire_data = Path('movies')

# Chemin vers les différents fichiers de données
chemin_movies = repertoire_data / 'movies.csv'
chemin_links = repertoire_data / 'links.csv'
chemin_ratings = repertoire_data / 'ratings.csv'
chemin_tags = repertoire_data / 'tags.csv' 

In [36]:
""" Cellule regroupant les fonctions pour la gestion de la base de donnée"""

def ouvrir_connection(base_de_donnee, utilisateur, mot_passe, host='localhost', port=5432):
    
    try:
        # création de l'objet conn
        conn = psycopg2.connect(dbname= base_de_donnee, user=utilisateur, password = mot_passe, host="localhost", port= 5432)

        conn.set_session(autocommit=True)   # on force
        
        # initialisation du cursor
        cursor = conn.cursor()
               
    except psycopg2.Error as e:
        print("Problème lors de la connection à la base de donnée")
        print(e)
        return None
    
    finally:
        cursor.close
        conn.close
    return conn
    
    
def lire_donnee(filename):
         
    #for row in file, boucle pour parcourir le fichier csv à la recherche des titres
    data = [] 
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:
            data.append(row)
                        
    return data

def couper_annee_titre_film(title):
    new_title=title.strip()
    year=None
    try:
        if new_title[-6]=='(' and new_title[-1] == ')':
            year=int(new_title[-5:-1])
            new_title=new_title[:-6].strip()
    except:
        pass
    return  new_title,year

def supprimer_table(conn, sql_suppression_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_suppression_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la suppression de la table")
        print(e)
        return
    cursor.close()
    print("La table a été supprimée avec succès")

    
def creer_table(conn, sql_creer_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creer_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

    
def inserer_donnees(conn, sql_inserer_donnee, donnees):
    try:
        cursor = conn.cursor()
        
        for d in donnees:
            titre_formate = couper_annee_titre_film(d[1])
            titre = titre_formate[0]
            year = titre_formate[1]
            cursor.execute(sql_inserer_donnee, (d[0], titre, d[2], year))
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

def lire_table(conn, sql_lire_donnee):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_lire_donnee)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)

    cursor.close()
    
    return data   

In [28]:
# SQL QUERY

sql_supprimer_table = """
    DROP TABLE IF EXISTS movie;
"""

sql_creer_table = """
    CREATE TABLE IF NOT EXISTS movie (
    movieid int primary key,
    title text,
    genres text,
    year int
    );
"""

sql_inserer_donnee = """
    INSERT INTO movie 
    (movieid, title, genres, year)
    VALUES (%s, %s, %s, %s);
"""

sql_lire_donnee = """
    SELECT *
    FROM movie;
"""

In [29]:
base_de_donnee = "movie2"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_pw')
    

In [30]:
print(ouvrir_connection)

<function ouvrir_connection at 0x0000027065EBBEE0>


In [31]:
conn = ouvrir_connection(base_de_donnee, utilisateur, mot_passe)

In [32]:
creer_table(conn, sql_creer_table)

La table a été crée avec succès


In [33]:
donnees = lire_donnee(chemin_movies)

In [34]:
inserer_donnees(conn, sql_inserer_donnee, donnees)

Les données ont été insérées avec succès


In [37]:
lire_table(conn, sql_lire_donnee)

Les données ont été lues avec succès


[(1, 'Toy Story', 'Adventure|Animation|Children|Comedy|Fantasy', 1995),
 (2, 'Jumanji', 'Adventure|Children|Fantasy', 1995),
 (3, 'Grumpier Old Men', 'Comedy|Romance', 1995),
 (4, 'Waiting to Exhale', 'Comedy|Drama|Romance', 1995),
 (5, 'Father of the Bride Part II', 'Comedy', 1995),
 (6, 'Heat', 'Action|Crime|Thriller', 1995),
 (7, 'Sabrina', 'Comedy|Romance', 1995),
 (8, 'Tom and Huck', 'Adventure|Children', 1995),
 (9, 'Sudden Death', 'Action', 1995),
 (10, 'GoldenEye', 'Action|Adventure|Thriller', 1995),
 (11, 'American President, The', 'Comedy|Drama|Romance', 1995),
 (12, 'Dracula: Dead and Loving It', 'Comedy|Horror', 1995),
 (13, 'Balto', 'Adventure|Animation|Children', 1995),
 (14, 'Nixon', 'Drama', 1995),
 (15, 'Cutthroat Island', 'Action|Adventure|Romance', 1995),
 (16, 'Casino', 'Crime|Drama', 1995),
 (17, 'Sense and Sensibility', 'Drama|Romance', 1995),
 (18, 'Four Rooms', 'Comedy', 1995),
 (19, 'Ace Ventura: When Nature Calls', 'Comedy', 1995),
 (20, 'Money Train', 'Action

-----------

## création table links, tags et ratings

In [ ]:
filename= 'movies csv/ratings.csv'

creation_query='''create table if not exists ratings(
                                       userId integer,
                                       movieId integer,
                                       rating float,
                                       timestamp integer,
                                       PRIMARY KEY (movieId, userId),
                                       FOREIGN KEY(movieId) REFERENCES movies(movieId)
                                       )'''

insert_query='insert into ratings(userId, movieId,rating, timestamp) values (?, ?, ?,?)'

executQuery(creation_query,insert_query,filename) 

In [ ]:
filename = 'movies csv/links.csv'

creation_query='''create table if not exists links(
                                       movieId integer,
                                       imdbId integer primary key,
                                       tmdbId integer,
                                       FOREIGN KEY(movieId) REFERENCES movies(movieId)
                                       ) '''

insert_query='insert into links (movieId, imdbId, tmdbId) values (?, ?, ?)'

executQuery(creation_query,insert_query,filename) 

In [ ]:
filename = 'movies csv/tags.csv'

creation_query=''' 
create table if not exists tags(
 movieId integer,
userId integer,
tag varchar,
timestamp integer,
PRIMARY KEY (movieId, userId,tag),
FOREIGN KEY(movieId) REFERENCES movies(movieId));'''

insert_query='insert into tags(userId, movieId,tag, timestamp) values (?, ?, ?,?)'

executQuery(creation_query,insert_query,filename) 